In [1]:
import sys
sys.path.append('../nimble_challenge')
sys.path.append('./ctpn/src/')
# sys.path.append('/home/arka_s/Caffe/cf/caffe/python/')
# import caffe
import two_imgs_eff
import pickle
from txt_data_storer import info_storer, info_storer_all
# import detectors
# import other

In [2]:
import skimage.io as skio
import skimage.draw as skd
%matplotlib
import os
import numpy as np

Using matplotlib backend: TkAgg


In [5]:
def get_cropped(img, bbox,default=True):
    '''
    Assuming format is (x1, y1, x2, y2)
    '''
    bbx = bbox.astype(int)
    if default:
        img1 = img[bbx[0]:bbx[2], bbx[1]:bbx[3], :]
    else:
        img1 = img[bbx[1]:bbx[3], bbx[0]:bbx[2], :]
    return img1

In [6]:
def get_img_with_bbx(img, rr, cc, copy=False, color=(255, 0, 0)):
    if copy:
        img1 = img.copy()
    else:
        img1 = img
    skd.set_color(img1, (rr, cc), color)
    return img1

In [7]:
def get_img_with_bbx2(img, actual_bbx2, default='co', copy=False, color=(255, 0, 0)):
    actual_bbx = actual_bbx2.copy()
    if 'co' in default:
        actual_bbx[0], actual_bbx[1] = actual_bbx[1], actual_bbx[0]
        actual_bbx[2], actual_bbx[3] = actual_bbx[3], actual_bbx[2]
    if 'wh' in default:
        actual_bbx[2] = actual_bbx[0] + actual_bbx[2]
        actual_bbx[3] = actual_bbx[1] + actual_bbx[3]
    abx = np.array((
    (actual_bbx[0], actual_bbx[1]),
    (actual_bbx[0], actual_bbx[3]),
    (actual_bbx[2], actual_bbx[3]),
    (actual_bbx[2], actual_bbx[1]),
    ))
    rr, cc = skd.polygon_perimeter(abx[:,0], abx[:,1], p_img.shape, clip=False)
    if copy:
        img1 = img.copy()
    else:
        img1 = img
    skd.set_color(img1, (rr, cc), color)
    return img1, get_cropped(img,actual_bbx)

In [8]:
tdir = '/home/nkovvuri/Rama_Work/dataset/Protest_Images/'
spdir = 'Modified_Images_ProtestL/'
swdir = 'Pruned_Protest_YFCCImages/'
npy_file = 'mod_fine_labels.npy'
pimg_list = os.listdir(tdir + spdir)
pimg_list = [i for i in pimg_list if i[-4:] == '.jpg' or i[-4:]=='.png']
test_img = pimg_list[0]
p_img = skio.imread(tdir + spdir + test_img)
actual_bbx = np.load(tdir + npy_file).item()[test_img[:-4]]
actual_bbx[0], actual_bbx[1] = actual_bbx[1], actual_bbx[0]
actual_bbx[2], actual_bbx[3] = actual_bbx[3], actual_bbx[2]
w_img_name = 'protest_img_' + test_img.split('_')[-1]
w_img = skio.imread(tdir + swdir + w_img_name)
abx = np.array((
    (actual_bbx[0], actual_bbx[1]),
    (actual_bbx[0], actual_bbx[3]),
    (actual_bbx[2], actual_bbx[3]),
    (actual_bbx[2], actual_bbx[1]),
))
rr, cc = skd.polygon_perimeter(abx[:,0], abx[:,1], p_img.shape, clip=True)
p_img_bbx = get_img_with_bbx(p_img, rr, cc, copy=True)
p_img_only_bbx = p_img[actual_bbx[0]:actual_bbx[2], actual_bbx[1]:actual_bbx[3]]
skio.imshow_collection([p_img_bbx, p_img_only_bbx, w_img])
# skio.imshow_collection([p_img_only_bbx, w_img])
# p_img_only_bbx.shape
# rr.shape

In [9]:
# Text Boxes
dict_bbx_mod = pickle.load(open('../../data/protest_data/dict_Modified_Images_ProtestL_text_lines.pkl'))
dict_bbx_prune = pickle.load(open('../../data/protest_data/dict_protest_text_lines.pkl'))
# skio.imshow(get_cropped(p_img, dict_bbx[test_img][1],default=False))
# skio.imshow(get_img_with_bbx2(p_img, dict_bbx[test_img][0], default='co', copy=True))
p_img_bbx2, pbbx = get_img_with_bbx2(p_img, dict_bbx_mod[test_img][0], default='co', copy=True)
# dict_bbx_prune[w_img_name][0].astype(int)
# skio.imshow(get_img_with_bbx2(w_img, dict_bbx_prune[w_img_name][0], default='co', copy=True))
w_img_bbx2, wbbx = get_img_with_bbx2(w_img, dict_bbx_prune[w_img_name][0], default='co', copy=True)
# skio.imshow(get_img_with_bbx2(w_img,np.array([112,  93, 159, 109,   0]),default=False,copy=True))
# skio.imshow([p_img_bbx2, w_img_bbx2])
# skio.imshow(p_img_bbx2)
# skio.imshow(w_img_bbx2)
# skio.imshow_collection([p_img_bbx2, w_img_bbx2]) # Working one
skio.imshow_collection([pbbx, p_img_bbx2])
skio.imshow_collection([wbbx, w_img_bbx2])
# skio.imsave('test_txt_img1.png', pbbx)
# skio.imsave('test_txt_img2.png', wbbx)
# bx = dict_bbx[test_img][1].astype(int)
# print(bx)
# p1 = p_img[bx[0]:bx[2], bx[1]:bx[3]]
# p1 = p_img[bx[1]:bx[3], bx[0]:bx[2]]
# skio.imshow(p1)
# dict_bbx[test_img][0].astype(int)
# dict_bbx_prune[w_img_name][0]

In [11]:
p_tl = dict_bbx_mod[test_img]
w_tl = dict_bbx_prune[w_img_name]
save_tdir = '../../data/protest_data/only_text/'
dest_pdir = save_tdir + test_img[:-4]
dest_wdir = save_tdir + w_img_name[:-4]
if not os.path.exists(dest_pdir):
    os.makedirs(dest_pdir)
if not os.path.exists(dest_wdir):
    os.makedirs(dest_wdir)
for ind, tl in enumerate(p_tl):
    p_img_bbx2, pbbx = get_img_with_bbx2(p_img, tl, default='co', copy=True)
    skio.imsave(dest_pdir +'/'+ str(ind) + '.png', pbbx)
for ind, tl in enumerate(w_tl):
    w_img_bbx2, wbbx = get_img_with_bbx2(w_img, tl, default='co', copy=True)
    skio.imsave(dest_wdir + '/' +str(ind) + '.png', wbbx)

In [48]:
# w_tl

In [35]:
# corr = two_imgs_eff.cmp_fv
help(skio.imsave)

Help on function imsave in module skimage.io._io:

imsave(fname, arr, plugin=None, **plugin_args)
    Save an image to file.
    
    Parameters
    ----------
    fname : str
        Target filename.
    arr : ndarray of shape (M,N) or (M,N,3) or (M,N,4)
        Image data.
    plugin : str
        Name of plugin to use.  By default, the different plugins are
        tried (starting with the Python Imaging Library) until a suitable
        candidate is found.  If not given and fname is a tiff file, the
        tifffile plugin will be used.
    
    Other parameters
    ----------------
    plugin_args : keywords
        Passed to the given plugin.



In [ ]:

# print(actual_bbx)
# p_img.shape
# poly = np.array((
#     (300, 300),
#     (480, 320),
#     (380, 430),
#     (220, 590),
#     (300, 300),
# ))
# rr, cc = polygon_perimeter(poly[:, 0], poly[:, 1], p_img.shape)

# p_img_bbx = p_img.copy()
# p_img_bbx[rr, cc, 0] = 255
# p_img_bbx[rr, cc, 1] = 0
# p_img_bbx[rr, cc, 2] = 0



In [ ]:
help(skd.set_color)

In [ ]:
print(abx)
poly

In [ ]:
# actual_bbx
# test_img.split('_')[-1]
actual_bbx

In [ ]:
from jupyter_core.paths import jupyter_config_dir
jupyter_dir = jupyter_config_dir()
jupyter_dir

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

from skimage.draw import (line, polygon, circle,
                          circle_perimeter,
                          ellipse, ellipse_perimeter,
                          bezier_curve, polygon_perimeter)


fig, ax1 = plt.subplots(ncols=1, nrows=1, figsize=(10, 6))


img = np.zeros((500, 500, 3), dtype=np.double)

# fill polygon
poly = np.array((
    (300, 300),
    (480, 320),
    (380, 430),
    (220, 590),
    (300, 300),
))
rr, cc = polygon_perimeter(poly[:, 0], poly[:, 1], img.shape)
img[rr, cc, 1] = 1


ax1.imshow(img)
ax1.set_title('No anti-aliasing')
ax1.axis('off')
